In [32]:
%load_ext autoreload
%autoreload 2

from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import uuid
from modules.db_operations import DatabaseConnector
from modules.sends import SendMail
EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet
# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

#Read in file
pd.set_option('display.max_columns', None)
df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\SQL_Scripts\email_prospects_csvs\baseball.csv')
df = df.drop_duplicates(subset='email')

def process(contact_column, sport, email_campaign_name, global_subject_line):

    data_list = []
    processed_emails = set()

    for index, row in df[:2].iterrows():

        # Get the current date
 
        #Set the time zone to Central Time
        central_time_zone = pytz.timezone('America/Chicago')
        now_central = datetime.now(central_time_zone)
        formatted_date = now_central.strftime("%m%d%Y%H%M%S")
        #When running campaigns to non schools
        try:
            school = row['School Name']
        except:
            school = None

        # png_path = row['PNG_PATH']
        contact = row[contact_column]
         #estiablish unique identifier, in order to match up FK relation, and merges
        unique_identifier = str(uuid.uuid4())

        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date, unique_identifier]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        SendMail.send(EMAIL_ADDRESS_FROM, EMAIL_PASS, global_subject_line, school, sport, contact, unique_identifier)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent', 'recipient_id'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date_sent'] = data_list['date_sent'].astype(str)

    data_list['subject'] = global_subject_line
    data_list['contact_position'] = contact_column
    data_list['sent_from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign_tag'] = email_campaign_name
    data_list['date_sent'] = now_central.strftime("%Y-%m-%d %H:%M:%S")

    data_list.to_csv('output.csv', index=False)

    return(data_list)

# --------------------------------------Running Process--------------------------

#Subject line references format string in sends.py, and is also the subject line of the email
global global_subject_line
global_subject_line = 'Majestic Baseball Apparel Round 2'

#Temporary testing to modify the Athletic Director email to my own
df.loc[0, 'email'] = '2015samtaylor@gmail.com'
df.loc[1, 'email'] = 'sammytaylor2006@yahoo.com'
df.loc[2, 'email'] = 'samuel.taylor@greendot.org'
df.loc[3, 'email'] = 'sammtaylor2006@gmail.com'
df.loc[3, 'email'] = 'jerrybons2006@gmail.com'

# #Must be the column name of the position wanting to send, sport fills in email headers, third arg is campaign name
# SCHOOL INTRO (contact_column, sport, email_campaign_name, global_subject_line)
#First arg is relevant for School Intro
#Second arg is for sport intro
#Third arg is for email campaign tag in the db
# Fourth arg is the Global subject line declared above. 


email_history = process('email' ,'Baseball', 'Majestic_Initial', global_subject_line)

# Instantiate the DatabaseConnector class, and send over the dataframe via append
db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
db_connector.send(email_history)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\modules\db_operations.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
